In [ ]:
# %pip install scipy
# %pip install matplotlib
# %pip install SpeechRecognition
# %pip install wave
%pip install dtw-python

In [ ]:
from dtw import * 
dtw
help(DTW)

Conversão para .WAV

In [ ]:
# # import required modules
# import subprocess

# # convert to wav file
# subprocess.call(['ffmpeg', '-i', 'racionais.mp3',
#                  'racionais.wav'])

Conversão audios estereos para mono

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile as wav

rate, data = wav.read('joao.wav')

# somando os dois canais e normalizando (passando de estereo para mono)
# data = np.sum(data, axis=1) / 2 
plt.plot(data)
plt.xlabel('Qtd. Amostras')
plt.ylabel('Amplitute')

Converter qtd. amostra para tempo (Eixo X)

In [ ]:
# encontrar o comprimento (total de amostras)
n = len(data)

# Frequencia de amostragem por sinal (quantas amostras por segundo)
fs = 48000 #48khz (audio) ou 44100 44.1khz (padrao spotify apple music) 
# https://emastered.com/pt/blog/sample-rate-for-audio#:~:text=A%20melhor%20taxa%20de%20amostragem,da%20sua%20onda%20sonora%20original.


T = 1/fs # tempo entre uma amostra e outra (inverso entre uma amostra e outra)

t = np.arange(0,n/fs,T) # (vetor de tempo) começa em 0seg, vai até o valor final do comprimento / frequencia de amostragem, e vai andando de T em T (valor calculado na variavel T)

plt.plot(t, data)
plt.xlabel('Tempo (s)')
plt.ylabel('Amplitute')

Transformada de Fourier

In [ ]:
from scipy.fft import fft, fftfreq
import numpy as np
import matplotlib.pyplot as plt

# saindo de segundos e indo para hz
f = fftfreq(n,T)

# numero complexo (transformada)
transf = fft(data)

# módulo - para tirar os números complexos (módulo)
transf = np.abs(transf)

plt.plot(f[f > 0],transf[f > 0]*1/n) # mascara para pegar apenas os sinais com frequencias maiores que zeros e normalização o eixo vertical (dividir a transformada pelo tamanho do sinal)
plt.xlabel('Frequência (Hz)')
plt.ylabel('Amplitute')

In [ ]:
chunksize = 1024
chunks = int(len(transf) / chunksize)
result = []

for j in range(chunks):
    arraychunk = np.zeros(chunksize)
    for i in range(chunksize):
        arraychunk[i] = transf[(j * chunksize) + i]
    result.append(arraychunk)
print(result[0])

In [ ]:
ranges = np.array([40, 80, 120, 180, 300])
fuzfactor = 2

result_length = len(result)
highscores = [[0] * len(ranges) for _ in range(result_length)]
points = [[0] * len(ranges) for _ in range(result_length)]

for t in range(result_length):
    for freq in range(40, 300):
        
        if result[t][freq] > 0:
            
            mag = np.log(np.abs(result[t][freq]) + 1)

            
            i = 0
            while ranges[i] < freq:
                i += 1
            index = i

           
            if mag > highscores[t][index]:
                highscores[t][index] = mag
                points[t][index] = freq
    
    
    h = (points[t][3] - (points[t][3] % fuzfactor)) * 100000000 + \
        (points[t][2] - (points[t][2] % fuzfactor)) * 100000 + \
        (points[t][1] - (points[t][1] % fuzfactor)) * 100 + \
        (points[t][0] - (points[t][0] % fuzfactor))
    
print(f"hashtag for music is: {h}")